In [17]:
import import_ipynb
from ElektronImFeld import ElektronImFeld
from EmptySmarti import ExampleSmarti
from ASmartis import *   
from Buttons import *   
from Chart2 import *
from ipywidgets import HBox, VBox, Layout
from math import pi, sin, cos
from IPython.display import display, clear_output
clear_output()

class Replay(ASmartis):
    
    def __init__(self,parameterList):
        
        super().__init__(parameterList)
        #self.updateTime = 0.5
        
        self.stopSendInformation()
        self.smartiDict ={"ElektronImFeld":ElektronImFeld,"ExampleSmarti":ExampleSmarti}
        
        self.createSmartiObjects()
        self.getCurrentSmarti()
        self.replayIndex = 0
        self.dataArray = []
        
       
        
        
        
    def createButtons(self):        
        
        #Erzeuge das Object, welches alle Buttons managt
        self.buttonsObject = Buttons(self)        
        
        #Erzeuge verschiedene Buttons        
        dropdown_smarti = self.buttonsObject.newDropdown("dd_smarti", [], "SmartI:")
        dropdown_sessionID = self.buttonsObject.newDropdown("dd_sessionID", [], "Session-ID:")
        button_runNumber = self.buttonsObject.newTogglePlayPause("b_runNumber", 30,30)  

     
        #Aktualisiere ein Dictionarie mit den Stati der Buttons        
        self.buttonStatesDict = self.buttonsObject.getStates()
        self.oldButtonStatesDict = dict(self.buttonStatesDict)
        self.buttonsDict = self.buttonsObject.getButtons()
        
        self.initalizeSmartiWidget()
        self.initalizeSessionIDWidget()
        
       
            
    def createChart(self):
        pass

        
    def createSmartiObjects(self):
        
        self.smartiObjectDict = {}
        for name in self.smartiNames:            
            s = name.split("-")
            smarti = s[0]
            arg = s[1].replace("(", "").replace(")", "")
            
            newSmarti = self.smartiDict[smarti](arg)    
            newSmarti.stopSendInformation()

            newButtonObject = newSmarti.getButtonObject()
            newButtons = newButtonObject.getButtons()

            for key in newButtons:
                #print(self.currentButtons[key])
                newButtonObject.disabled(key,True)
            
            self.smartiObjectDict[name] = newSmarti
            
            
    def visualize(self): 
        
        clear_output()        
        
        #Darstellung   
        display(HBox([self.buttonsDict["dd_smarti"],self.buttonsDict["dd_sessionID"],self.buttonsDict["b_runNumber"]]))
        self.currentSmarti.visualize()

    
    
    def getCurrentSmarti(self):
        
        self.currentSmarti = self.smartiObjectDict[self.buttonStatesDict["dd_smarti"]]
            
        self.currentButtonObject = self.currentSmarti.getButtonObject()
        self.currentButtons = self.currentButtonObject.getButtons()
        
        self.currentSmarti.start()
            
        

     
    def update(self):
        
        if self.oldButtonStatesDict["dd_smarti"]!= self.buttonStatesDict["dd_smarti"]:
            self.currentSmarti.stop()
            #self.currentSmarti.reset()
            
            self.getCurrentSmarti()
            self.visualize()
            self.initalizeSessionIDWidget()
        if self.oldButtonStatesDict["b_runNumber"]!= self.buttonStatesDict["b_runNumber"]:
            if self.buttonStatesDict["dd_sessionID"]!= None:
                self.dataArray = self.server.getSession(self.buttonStatesDict["dd_smarti"],self.buttonStatesDict["dd_sessionID"])
                #self.currentSmarti.reset()
                
                self.replayIndex = 0
                
                self.startReplay()
                self.buttonsObject.changeState("b_runNumber",False)
            else:
                self.buttonsObject.changeState("b_runNumber",False)
                
                
    def initalizeSmartiWidget(self):
        
        self.smartiNames = self.server.getAllSmartiNames()
        self.buttonsObject.changeOptions("dd_smarti",self.smartiNames)
     
    
    def initalizeSessionIDWidget(self):
        
        sessionID = self.server.getSessionIDofSmarti(self.buttonStatesDict["dd_smarti"])
        self.buttonsObject.changeOptions("dd_sessionID",sessionID)
        
        
    def startReplay(self):
        
        #Array Positions:
        sessionIDPosition = 0
        smartiPosition = 1
        smartiArgumentsPosition = 2
        timePosition = 3
        buttonPosition = 4
        valuePosition = 5
        dataLength = len(self.dataArray)
        i = self.replayIndex
        currentData = self.dataArray[i]
        
             
        self.currentButtonObject.changeState(currentData[buttonPosition],self.castingValue(currentData[valuePosition]))
        #print(currentData[buttonPosition],self.castingValue(currentData[valuePosition]))
        currentTime = datetime.strptime(currentData[timePosition], "%m/%d/%Y, %H:%M:%S.%f")-timedelta(seconds=0)
        
        
        if i < dataLength-1:
            nextTime = datetime.strptime(self.dataArray[i+1][timePosition], "%m/%d/%Y, %H:%M:%S.%f")-timedelta(seconds=0)
            timeDiff = nextTime-currentTime
            #print(timeDiff.total_seconds())    
            threading.Timer(round(timeDiff.total_seconds(),3), self.startReplay).start()
            self.replayIndex+=1
        
        
    def castingValue(self,value):
        
        if value == "True":
            return bool(value)
        elif value == "False":
            return False
        else:
            try:
                a = float(value)
                b = int(value)
                if a != b:
                    return a
                else:
                    return b
            except:
                #raise ValueError("failed to coerce str to int or float")
                return value
            
            
    def clearFile(self):
        
        self.server.clearFile()

           
        
    
            

In [18]:
r = Replay(None)
r.visualize()
r.start()



In [15]:
r.clearFile()

In [3]:
e.stop()